In [1]:
import numpy as np
import pandas as pd

import os

os.getcwd()

data = pd.read_csv("/Users/siddharthrekhade/Downloads/Mobile Phone Pricing.csv")

print(data.head())

   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0            842     0          2.2         0   1       0           7    0.6   
1           1021     1          0.5         1   0       1          53    0.7   
2            563     1          0.5         1   2       1          41    0.9   
3            615     1          2.5         0   0       0          10    0.8   
4           1821     1          1.2         0  13       1          44    0.6   

   mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  talk_time  \
0        188        2  ...         20       756  2549     9     7         19   
1        136        3  ...        905      1988  2631    17     3          7   
2        145        5  ...       1263      1716  2603    11     2          9   
3        131        6  ...       1216      1786  2769    16     8         11   
4        141        2  ...       1208      1212  1411     8     2         15   

   three_g  touch_screen  wifi  price_

In [2]:
from sklearn.preprocessing import MinMaxScaler

preprocessor = MinMaxScaler()
features = ["battery_power", "clock_speed", "int_memory", "m_dep", "mobile_wt", "n_cores", "px_height", "px_width", "ram", "sc_w", "talk_time"]

data[features] = preprocessor.fit_transform(data[features])

print(data.head())

   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  \
0       0.227789     0         0.68         0   1       0    0.080645   
1       0.347361     1         0.00         1   0       1    0.822581   
2       0.041416     1         0.00         1   2       1    0.629032   
3       0.076152     1         0.80         0   0       0    0.129032   
4       0.881764     1         0.28         0  13       1    0.677419   

      m_dep  mobile_wt   n_cores  ...  px_height  px_width       ram  sc_h  \
0  0.555556   0.900000  0.142857  ...   0.010204  0.170895  0.612774     9   
1  0.666667   0.466667  0.285714  ...   0.461735  0.993324  0.634687    17   
2  0.888889   0.541667  0.571429  ...   0.644388  0.811749  0.627205    11   
3  0.777778   0.425000  0.714286  ...   0.620408  0.858478  0.671566    16   
4  0.555556   0.508333  0.142857  ...   0.616327  0.475300  0.308658     8   

       sc_w  talk_time  three_g  touch_screen  wifi  price_range  
0  0.388889   0.944444   

In [3]:
from sklearn.model_selection import train_test_split

X = data.drop(columns = ["price_range"])
y = data["price_range"]
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 23)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train.values, dtype = torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype = torch.float32)
y_train_tensor = torch.tensor(y_train.values.squeeze(), dtype = torch.long)
y_test_tensor = torch.tensor(y_test.values.squeeze(), dtype = torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 64)

In [5]:
class PredictionNNModel(nn.Module):
    def __init__(self, input_dim, hidden1_dim, hidden2_dim, output_dim):
        super(PredictionNNModel, self).__init__()

        self.fc1 = nn.Linear(input_dim, hidden1_dim)
        self.fc2 = nn.Linear(hidden1_dim, hidden2_dim)
        self.fc3 = nn.Linear(hidden2_dim, output_dim)

    def forward(self, x):
        return self.fc3(self.fc2(self.fc1(x)))

input_dim = 20
hidden1_dim = 64
hidden2_dim = 32
output_dim = 4

model = PredictionNNModel(input_dim, hidden1_dim, hidden2_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

In [8]:
num_epochs = 200
model.train()

for epoch in range(num_epochs):
    epoch_loss = 0
    correct = 0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == batch_y).sum().item()

    acc = correct / len(train_loader.dataset)
    print(f"Epoch {epoch+1} | Loss: {epoch_loss:.4f} | Accuracy: {acc:.4f}")

Epoch 1 | Loss: 9.4013 | Accuracy: 0.8807
Epoch 2 | Loss: 9.3406 | Accuracy: 0.8836
Epoch 3 | Loss: 9.3385 | Accuracy: 0.8800
Epoch 4 | Loss: 9.2511 | Accuracy: 0.8843
Epoch 5 | Loss: 9.2058 | Accuracy: 0.8843
Epoch 6 | Loss: 9.1489 | Accuracy: 0.8857
Epoch 7 | Loss: 9.1466 | Accuracy: 0.8836
Epoch 8 | Loss: 9.0513 | Accuracy: 0.8843
Epoch 9 | Loss: 9.0406 | Accuracy: 0.8879
Epoch 10 | Loss: 8.9715 | Accuracy: 0.8900
Epoch 11 | Loss: 8.9382 | Accuracy: 0.8879
Epoch 12 | Loss: 8.8834 | Accuracy: 0.8943
Epoch 13 | Loss: 8.8323 | Accuracy: 0.8914
Epoch 14 | Loss: 8.8107 | Accuracy: 0.8914
Epoch 15 | Loss: 8.7536 | Accuracy: 0.8907
Epoch 16 | Loss: 8.7037 | Accuracy: 0.8907
Epoch 17 | Loss: 8.7035 | Accuracy: 0.8857
Epoch 18 | Loss: 8.6127 | Accuracy: 0.8957
Epoch 19 | Loss: 8.5725 | Accuracy: 0.8864
Epoch 20 | Loss: 8.5767 | Accuracy: 0.8979
Epoch 21 | Loss: 8.4891 | Accuracy: 0.8950
Epoch 22 | Loss: 8.4953 | Accuracy: 0.8929
Epoch 23 | Loss: 8.4248 | Accuracy: 0.8921
Epoch 24 | Loss: 8.3

In [9]:
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    preds = torch.argmax(outputs, dim=1)
    acc = (preds == y_test_tensor).float().mean()
    print(f"Accuracy: {acc.item():.4f}")

Accuracy: 0.9350
